In [12]:
using LinearAlgebraicRepresentation
using BenchmarkTools

In [13]:
using LinearAlgebraicRepresentation
Lar=LinearAlgebraicRepresentation
function frag_face_channel(in_chan, out_chan, V, EV, FE, sp_idx)
    run_loop = true
    while run_loop
        
        sigma = take!(in_chan)
        
        if sigma != -1
            put!(out_chan,Lar.Arrangement.frag_face(V, EV, FE, sp_idx, sigma))
        
    
            
        else
            run_loop = false
        end
    end
    return V,EV
end



frag_face_channel (generic function with 1 method)

In [16]:

using Distributed
using BenchmarkTools
using SparseArrays
using LinearAlgebraicRepresentation

Lar=LinearAlgebraicRepresentation
 
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])

fs_num = size(FE, 1)
    
FV = Lar.compute_FV( EV, FE )
model = (convert(Lar.Points,V'), FV)
sp_idx = Lar.spaceindex(model)

in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(10))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
    end
end

@time frag_face_channel(in_chan,out_chan,Lar.Points(V),EV,FE,sp_idx)



  0.021266 seconds (69.41 k allocations: 3.840 MiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1)

Task (runnable) @0x0000000014ba73d0

Channel{Tuple}(sz_max:0,sz_curr:0)